<h1 style="text-align: center">
    Qu'est ce que le SVC ?

</h1>



* Basé sur le SVM (Support Vector Machine, ou Machine à Vecteurs de Support en français), c'est un modèle de classification fonctionnant en identifiant un hyperplan (droite en 2D ou surface en 3D) de séparation optimale entre différentes classes (features) dans l'espace. 
Il peut être utilisé aussi bien pour la classification binaire que multi-classe.

<hr>

* Le SVC (Support Vector Classification, ou Classification par Vecteurs de Support en français) est une implémentation de l'algorithme SVM pour la classification. 
Il utilise un noyau (kernel) pour transformer les données d'entrée dans un espace de features de plus haute dimension, de manière à ce qu'un hyperplan de séparation puisse être trouvé facilement.
Grossièrement le noyau correspond à la fonction générée pour produire l'hyperplan (droite ou plan).

<hr>

* Le SVM est souvent utilisé lorsque les données sont "linéairement séparable", c'est-à-dire qu'il existe un hyperplan qui peut séparer clairement les différentes classes les unes des autres. 
Si les données ne sont pas linéairement séparables, le SVM peut quand même être utilisé en utilisant un noyau qui transforme les données dans un espace de features de plus haute dimension où elles deviennent linéairement séparables.

<hr>

* Le SVM est souvent considéré comme un modèle de classification robuste et performant, en particulier lorsque les données ont un nombre élevé de dimensions (c'est-à-dire sont à "haute dimension"). 
Cependant, il peut être relativement lent à entraîner lorsque les données ont un grand nombre d'exemples (c'est-à-dire sont à "grande échelle").


<h1 style="text-align: center">
     Ci-dessous une liste des hyperparamètres les plus courament utilisés pour optimiser le modèle de SVC:
     
</h1>

* C : ce paramètre contrôle la force de régularisation du modèle. 
Plus C est grand, plus le modèle sera complexe et donc plus il y a de chances qu'il surapprenne (c'est-à-dire qu'il s'ajuste trop bien aux données d'entraînement au détriment de sa généralisation aux données de test). 
Un C faible signifie une régularisation forte et un modèle moins complexe.

<hr>

* kernel : ce paramètre spécifie quel type de noyau utiliser pour transformer les données d'entrée dans un espace de features de plus haute dimension. Les options courantes incluent "linear" (pour un noyau linéaire), "rbf" (p our un noyau gaussien, aussi appelé "radial basis function"), et "poly" (pour un noyau polynomial).
  
<hr>

* degree : ce paramètre est utilisé lorsque le noyau "poly" est sélectionné et spécifie le degré du polynôme à utiliser.

<hr>

* gamma : ce paramètre est utilisé lorsque le noyau "rbf" ou "poly" est sélectionné et contrôle la complexité du modèle. Plus gamma est grand, plus le modèle sera complexe et donc plus il y a de chances qu'il surapprenne.

<hr>

* coef0 : ce paramètre est utilisé lorsque le noyau "poly" est sélectionné et contrôle le degré de liberté du modèle.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [2]:
df = pd.read_csv("mushrooms.csv")

In [3]:
df.sample(3)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
3104,p,x,s,g,f,c,f,w,n,g,...,s,w,w,p,w,o,p,k,s,d
3880,e,f,y,e,t,n,f,c,b,p,...,s,g,g,p,w,o,p,k,y,d
2156,e,x,y,n,t,n,f,c,b,n,...,s,p,g,p,w,o,p,n,y,d


In [4]:
# Split the dataset into features (X) and labels (y)
y = df["class"]
X = df[["gill-size", "gill-color"]]

In [5]:
X = X.values
X

array([['n', 'k'],
       ['b', 'k'],
       ['b', 'n'],
       ...,
       ['b', 'n'],
       ['n', 'b'],
       ['b', 'y']], dtype=object)

In [6]:
encoder = OrdinalEncoder()
X = encoder.fit_transform(X)

In [7]:
X

array([[ 1.,  4.],
       [ 0.,  4.],
       [ 0.,  5.],
       ...,
       [ 0.,  5.],
       [ 1.,  0.],
       [ 0., 11.]])

In [8]:
encoder_2 = OrdinalEncoder(categories=[["p"], ["e"]])
df["class"] = encoder.fit_transform(df[["class"]])
y = df["class"]

In [9]:
y.to_numpy()

array([1., 0., 0., ..., 0., 1., 0.])

In [11]:
for i in range(1, 200, 50) :  
    models = (
        svm.SVC(kernel="linear", C=float(i)),
        svm.LinearSVC(C=float(i), max_iter=500),
        svm.SVC(kernel="rbf", gamma=0.7, C=float(i)),
        svm.SVC(kernel="poly", degree=3, gamma="auto", C=float(i)),
    )
    models = (clf.fit(X, y) for clf in models)
    titles = (
        f"SVC kernel lineaire (C={i})",
        f"LinearSVC kernel lineaire (C={i})",      
        f"SVC kernel RBF (C={i})",
        f"SVC kernel polynôme degrée 3 (C={i})",
    )
    # Set-up grille 2x2 pour le plot.
    fig, sub = plt.subplots(2, 2)
    plt.subplots_adjust(wspace=0.4, hspace=0.4)

    X0, X1 = X[:, 0], X[:, 1]

    for clf, title, ax in zip(models, titles, sub.flatten()):
        disp = DecisionBoundaryDisplay.from_estimator(
            clf,
            X,
            response_method="predict",
            cmap=plt.cm.coolwarm,
            alpha=0.8,
            ax=ax,
            xlabel="gill-size",
            ylabel="gill-color",
        )
        ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors="k")
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_title(title)

C:\Users\mathi\AppData\Roaming\Python\Python311\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\mathi\AppData\Roaming\Python\Python311\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
plt.show()